In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import IFrame, display
InteractiveShell.ast_node_interactivity = "all"

# Probabilities to cluster algorithm

A notebook to hash out this algorithm and check it works.

Will hopefully turn into a unit test too, hence CVSs into version control.

In [2]:
from src import locations as loc

import pandas as pd
import duckdb
from pathlib import Path

In [3]:
clus = pd.read_csv(Path(loc.PROJECT_DIR, "test", "clusters.csv"))
prob = pd.read_csv(Path(loc.PROJECT_DIR, "test", "probabilities.csv"))
val = pd.read_csv(Path(loc.PROJECT_DIR, "test", "validate.csv"))

### Parallel

Sometimes we might join several tables to `probabilities` at once, then add them to `clusters` together.

In [120]:
clus_init = duckdb.sql("""
    drop sequence if exists uuid;
    drop sequence if exists cluster;
    create sequence uuid start 1;
    create sequence cluster start 1;
    select
        nextval('uuid') as uuid,
        nextval('cluster') as cluster,
        id,
        source,
        0 as n,
    from
        prob
    where
        cluster = 0
""")
clus_init

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [121]:
clus_complete = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob
        anti join clus_init cl on
            cl.id = prob.id
            and cl.source = prob.source
        where 
            probability > 0.7
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_init
""")

In [122]:
clus_check_l = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n::int as n
    from
        clus_complete
    order by
        cluster,
        source,
        id,
        n
""")
clus_check_r = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n_par::int as n
    from
        clus
    order by
        cluster,
        source,
        id,
        n_par
""")
clus_check_l.df().equals(clus_check_r.df())
clus_check_l.df().compare(clus_check_r.df())

True

Empty DataFrame
Columns: []
Index: []

### Sequential

Sometimes we'll add one table to `probabilities`, then resolve to `clusters`, then do that over and over.

In [132]:
clus_init = duckdb.sql("""
    drop sequence if exists uuid;
    drop sequence if exists cluster;
    create sequence uuid start 1;
    create sequence cluster start 1;
    select
        nextval('uuid') as uuid,
        nextval('cluster') as cluster,
        id,
        source,
        0 as n,
    from
        prob
    where
        cluster = 0
""")
clus_init

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [133]:
prob_n1 = duckdb.sql("""
    select
        *
    from
        prob
    where
        source = 2
""")
prob_n2 = duckdb.sql("""
    select
        *
    from
        prob
    where
        source = 3
""")
prob_n3 = duckdb.sql("""
    select
        *
    from
        prob
    where
        source = 4
""")

In [134]:
clus_n1 = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob_n1 prob
        anti join clus_init cl on
            cl.id = prob.id
            and cl.source = prob.source
        where
            probability > 0.7
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_init
""")
clus_n1

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
│     4 │       1 │ will_inc_t2  │      2 │     1 │
│     5 │       4 │ pedro_inc_t2 │      2 │     1 │
│     6 │       2 │ leo_inc_t2   │      2 │     1 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [135]:
clus_n2 = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        2 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob_n2 prob
        anti join clus_n1 cl on
            cl.id = prob.id
            and cl.source = prob.source
        where
            probability > 0.7
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_n1
""")
clus_n2

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
│    10 │       1 │ will_inc_t2  │      2 │     1 │
│    11 │       4 │ pedro_inc_t2 │      2 │     1 │
│    12 │       2 │ leo_inc_t2   │      2 │     1 │
│    13 │       1 │ will_inc_t3  │      3 │     2 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [136]:
clus_n3 = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        3 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob_n3 prob
        anti join clus_n2 on
            clus_n2.id = prob.id
            and clus_n2.source = prob.source
        where
            probability > 0.7
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_n2
""")
clus_n3

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
│    14 │       1 │ will_inc_t2  │      2 │     1 │
│    15 │       4 │ pedro_inc_t2 │      2 │     1 │
│    16 │       2 │ leo_inc_t2   │      2 │     1 │
│    27 │       1 │ will_inc_t3  │      3 │     2 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [137]:
clus_check_l1 = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n::int as n
    from
        clus_n2
    order by
        cluster,
        source,
        id,
        n
""")
clus_check_l2 = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n::int as n
    from
        clus_n3
    order by
        cluster,
        source,
        id,
        n
""")
clus_check_r = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n_seq::int as n
    from
        clus
    order by
        cluster,
        source,
        id,
        n_par
""")
clus_check_l1.df().equals(clus_check_r.df())
clus_check_l1.df().compare(clus_check_r.df())
clus_check_l2.df().equals(clus_check_r.df())
clus_check_l2.df().compare(clus_check_r.df())

True

Empty DataFrame
Columns: []
Index: []

True

Empty DataFrame
Columns: []
Index: []